In [1]:
import train as proteo_train
import os
import torch
import torch.nn.functional as F
import torch_geometric
from proteo.datasets.ftd import FTDDataset, reverse_log_transform
import torch.nn.functional as F
import pytorch_lightning as pl
from scipy.stats import zscore
from proteo.checkpoint_analysis import full_load_and_run_and_convert, process_checkpoints
%load_ext autoreload
%autoreload 2

In [4]:
device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
csf_all_mean = 2.124088581365514
csf_all_std = 0.8733420033790319

checkpoint_0 = "/home/lcorneli"

checkpoint_1 = "/data/lcornelis/outputs/ray_results/TorchTrainer_2024-10-16_16-32-34/model=gat-v4,seed=6021_97_act=tanh,adj_thresh=0.5000,batch_size=32,dropout=0,l1_lambda=0.0000,lr=0.0023,lr_scheduler=LambdaLR,moda_2024-10-16_16-32-35/checkpoint_000050"

module, config, train_preds, train_targets, train_mse, val_preds, val_targets, val_mse = full_load_and_run_and_convert(checkpoint_1,device, csf_all_mean, csf_all_std)

accumulate_grad_batches=1 trainer_accelerator='gpu' act='tanh' log_every_n_steps=10 use_master_nodes=False lr=0.002255069173338685 wandb_offline=False gat_v4_fc_dropout=[0.1, 0.2, 0.5] precision='32-true' gat={'num_layers': 2, 'hidden_channels': 256, 'heads': 4, 'v2': True} sync_batchnorm=False use_progress_bar=True num_nodes_choices=[7258] gat-v4={'hidden_channels': [64, 128], 'heads': [2, 2], 'use_layer_norm': True, 'which_layer': ['layer1', 'layer2', 'layer3', 'sex', 'mutation'], 'fc_dim': [256, 512, 512, 128], 'fc_dropout': 0.2, 'fc_act': 'tanh', 'weight_initializer': 'orthogonal', 'num_layers': None} output_dir='/home/lcornelis/code/proteo/outputs' lr_scheduler_choices=['LambdaLR', 'ReduceLROnPlateau', 'ExponentialLR', 'StepLR', 'CosineAnnealingLR'] mutation_choices=[['GRN', 'MAPT', 'C9orf72', 'CTL']] grace_period=30 weight_decay=0 y_val='nfl' gat_v4_fc_act=['relu', 'tanh', 'sigmoid', 'leaky_relu', 'elu'] gat_v4_fc_dim=[[64, 128, 128, 32], [128, 256, 256, 64], [256, 512, 512, 128]

In [49]:
#(F.mse_loss(val_preds[:16], val_targets[:16]).item() + F.mse_loss(val_preds[16:32], val_targets[16:32]).item()+F.mse_loss(val_preds[32:], val_targets[32:]).item())/2
(F.mse_loss(val_preds[:32], val_targets[:32]).item() +F.mse_loss(val_preds[32:], val_targets[32:]).item())/2

0.30044277757406235

In [36]:
val_preds[32:].shape

torch.Size([13, 1])

In [21]:
import train as proteo_train

In [22]:
train_dataset, test_dataset = proteo_train.construct_datasets(config)
train_loader, test_loader = proteo_train.construct_loaders(config,train_dataset,test_dataset)
test_dataset.data

Loading datasets from /data/lcornelis/data/data_louisa
Absolute path: /data/lcornelis/data/data_louisa
Directory contents: ['processed', 'raw']
Processed file names: ['ftd_y_val_nfl_adj_thresh_0.7_num_nodes_7258_mutation_GRN,MAPT,C9orf72,CTL_csf_sex_M,F_masternodes_False_sex_specific_True_train.pt', 'ftd_y_val_nfl_adj_thresh_0.7_num_nodes_7258_mutation_GRN,MAPT,C9orf72,CTL_csf_sex_M,F_masternodes_False_sex_specific_True_test.pt']
Loading data from: /data/lcornelis/data/data_louisa/processed/ftd_y_val_nfl_adj_thresh_0.7_num_nodes_7258_mutation_GRN,MAPT,C9orf72,CTL_csf_sex_M,F_masternodes_False_sex_specific_True_train.pt
Train dataset loaded successfully
Processed file names: ['ftd_y_val_nfl_adj_thresh_0.7_num_nodes_7258_mutation_GRN,MAPT,C9orf72,CTL_csf_sex_M,F_masternodes_False_sex_specific_True_train.pt', 'ftd_y_val_nfl_adj_thresh_0.7_num_nodes_7258_mutation_GRN,MAPT,C9orf72,CTL_csf_sex_M,F_masternodes_False_sex_specific_True_test.pt']
Loading data from: /data/lcornelis/data/data_loui

Data(x=[326610, 1], edge_index=[2, 506010], y=[45], sex=[45], mutation=[45], age=[45])

In [26]:
val_preds, val_targets = [], []
for batch in test_loader:
    batch.to(device)
    # Forward pass
    # print(module(batch))
    pred = module(batch)
    target = batch.y.view(pred.shape)

    # Store predictions and targets
    val_preds.append(pred.cpu())
    val_targets.append(target.cpu())

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/lcornelis/anaconda3/envs/proteo/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/lcornelis/anaconda3/envs/proteo/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/lcornelis/anaconda3/envs/proteo/lib/python3.11/site-packages/torch_geometric/loader/dataloader.py", line 27, in __call__
    return Batch.from_data_list(
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/lcornelis/anaconda3/envs/proteo/lib/python3.11/site-packages/torch_geometric/data/batch.py", line 97, in from_data_list
    batch, slice_dict, inc_dict = collate(
                                  ^^^^^^^^
  File "/home/lcornelis/anaconda3/envs/proteo/lib/python3.11/site-packages

In [24]:
module.validation_step(test_dataset.data)

RuntimeError: Given normalized_shape=[7258], expected input with shape [*, 7258], but got input of size[1, 326610]

In [ ]:
"/home/lcornelis/code/proteo/outputs/ray_results/TorchTrainer_2024-10-17_16-51-50/model=gat-v4,seed=1812_148_act=leaky_relu,adj_thresh=0.7000,batch_size=32,dropout=0.1000,l1_lambda=0.0000,lr=0.0044,lr_scheduler=L_2024-10-17_16-51-51/checkpoint_000094"
"/home/lcornelis/code/proteo/outputs/ray_results/TorchTrainer_2024-10-17_16-51-50/model=gat-v4,seed=1812_148_act=leaky_relu,adj_thresh=0.7000,batch_size=32,dropout=0.1000,l1_lambda=0.0000,lr=0.0044,lr_scheduler=L_2024-10-17_16-51-51/checkpoint_000094"

In [3]:
# Sex_specific_adj no sex
checkpoint1 = "/home/lcornelis/code/proteo/outputs/ray_results/TorchTrainer_2024-10-15_15-51-21/model=gat-v4,seed=15680_180_act=tanh,adj_thresh=0.3500,batch_size=32,dropout=0.1000,l1_lambda=0.0007,lr=0.0005,lr_scheduler=Lambda_2024-10-15_15-51-23/checkpoint_000001"
checkpoint2 = "/home/lcornelis/code/proteo/outputs/ray_results/TorchTrainer_2024-10-15_15-51-21/model=gat-v4,seed=60527_226_act=tanh,adj_thresh=0.9000,batch_size=32,dropout=0,l1_lambda=0.0025,lr=0.0006,lr_scheduler=LambdaLR,mo_2024-10-15_17-14-43/checkpoint_000006"
full_load_and_run_and_convert(checkpoint1 ,device, csf_all_mean, csf_all_std)

adj_thresh_choices=[0.1, 0.2, 0.35, 0.5, 0.7, 0.9] dropout=0.1 gat-v4={'hidden_channels': [8, 16], 'heads': [4, 4], 'use_layer_norm': True, 'which_layer': ['layer1', 'layer2', 'layer3', 'sex', 'mutation'], 'fc_dim': [128, 256, 256, 64], 'fc_dropout': 0.1, 'fc_act': 'relu', 'weight_initializer': 'orthogonal', 'num_layers': None} gat_hidden_channels=[8, 32, 128, 256] output_dir='/home/lcornelis/code/proteo/outputs' lr_min=1e-06 gcn_num_layers=[2, 3, 4] lr_scheduler='LambdaLR' nodes_count=1 raw_file_name='ALLFTD_dataset_for_nina_louisa_071124_age_adjusted.csv' gpu_per_worker=1 gcn={'num_layers': 3, 'hidden_channels': 32} num_workers=16 trainer_accelerator='gpu' use_progress_bar=True gat_num_layers=[2, 4, 6, 12] modality_choices=['csf'] batch_size=32 seed=15680 lr_scheduler_choices=['LambdaLR', 'ReduceLROnPlateau', 'ExponentialLR', 'StepLR', 'CosineAnnealingLR'] gat_heads=[1, 2, 4, 8] reduction_factor=6 num_nodes=7258 lr=0.0005472969709054848 master_nodes=['sex', 'mutation', 'age'] epochs=

TypeError: expected 0 arguments, got 1

In [5]:
#Sanity check
def compute_manual_mse(val_preds, val_targets):
    """
    Manually computes the Mean Squared Error (MSE) for the given predictions and targets.

    Parameters:
    val_preds (list of list of torch.Tensor): The predicted values.
    val_targets (list of list of torch.Tensor): The true target values.

    Returns:
    float: The computed Mean Squared Error.
    """
   # Compute the squared differences
    squared_diffs = (val_preds - val_targets) ** 2

    # Compute the mean of the squared differences
    mse = squared_diffs.mean().item()

    return mse

print(compute_manual_mse(val_preds, val_targets))

0.2644430100917816


In [13]:
import os
import torch
import torch.nn.functional as F
import train as proteo_train

# Define a function to load the checkpoint and calculate MSE
def load_checkpoint_and_calculate_mse(relative_checkpoint_path, levels_up=5):
    # Get the current script directory
    current_directory = os.getcwd()
    
    # Navigate up the specified number of levels
    for _ in range(levels_up):
        current_directory = os.path.dirname(current_directory)
    
    # Construct the full path to the checkpoint
    checkpoint_path = os.path.join(current_directory, relative_checkpoint_path)
    print(f"Loading checkpoint from: {checkpoint_path}")

    # Check if the file exists to avoid errors
    if not os.path.isfile(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint file not found: {checkpoint_path}")

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    print("Checkpoint keys:", checkpoint.keys())
    print("checkpoint state_dict keys:", checkpoint['state_dict'].keys())

    module = proteo_train.Proteo.load_from_checkpoint(checkpoint_path)

    # Access the attributes
    # best_val_pred = module.best_val_pred
    # print("best_val_pred:", best_val_pred)
    # # print("min_val loss:", module.val_loss)
    # best_val_target = module.best_val_target
    # best_train_pred = module.best_train_pred
    # best_train_target = module.best_train_target

    # # Calculate MSE for validation and training
    # mse_val = F.mse_loss(best_val_pred, best_val_target).item()
    # mse_train = F.mse_loss(best_train_pred, best_train_target).item()

    return module, checkpoint

# Example usage
relative_checkpoint_path = '/scratch/lcornelis/outputs/ray_results/TorchTrainer_2024-07-31_16-47-02/model=gat-v4,seed=19543_0_act=relu,adj_thresh=0.1000,batch_size=8,dropout=0,l1_lambda=0.0000,lr=0.1000,lr_scheduler=LambdaLR,modal_2024-07-31_16-47-02/checkpoint_000001/checkpoint.cpkt'
module, checkpoint = load_checkpoint_and_calculate_mse(relative_checkpoint_path)
# print(f"MSE Loss for validation set: {mse_val}")
# print(f"MSE Loss for training set: {mse_train}")


Loading checkpoint from: /scratch/lcornelis/outputs/ray_results/TorchTrainer_2024-07-31_15-31-35/model=gat-v4,seed=19543_0_act=relu,adj_thresh=0.1000,batch_size=8,dropout=0,l1_lambda=0.0000,lr=0.1000,lr_scheduler=LambdaLR,modal_2024-07-31_15-31-36/checkpoint_000001/checkpoint.ckpt
Checkpoint keys: dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])
checkpoint state_dict keys: odict_keys(['model.convs.0.att_src', 'model.convs.0.att_dst', 'model.convs.0.bias', 'model.convs.0.lin.weight', 'model.convs.1.att_src', 'model.convs.1.att_dst', 'model.convs.1.bias', 'model.convs.1.lin.weight', 'model.pools.0.weight', 'model.pools.0.bias', 'model.pools.1.weight', 'model.pools.1.bias', 'model.layer_norm.weight', 'model.layer_norm.bias', 'model.encoder.0.0.weight', 'model.encoder.0.0.bias', 'model.encoder.1.0.weight', 'model.encoder.1.0.bias', 'model.encoder.2.0.weight', 'model.